---
# 환경 설정
---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers==4.3.0
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 14.9 MB/s 
     |████████████████████████████████| 880 kB 82.2 MB/s 
     |████████████████████████████████| 3.3 MB 46.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=7566e98ab85ce722fa571d3d0c28a7f8bd65dda8a483ff85860bd59c18a4c661
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# pip install transformers-interpret

In [5]:
import wandb

wandb.login()
wandb.init()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: glad415. Use `wandb login --relogin` to force relogin


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import re
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [8]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [9]:
# GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

bertmodel = AutoModel.from_pretrained("/content/drive/MyDrive/Kobert/TAPT_Model_Save")

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/Kobert/TAPT_Model_Save and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---
# sweep config 정의
---

In [11]:
sweep_config = {
    'name' : 'klue_bert_TAPT_before',
    'method': 'random',
    'metric' : {
        'name': 'Test_acc',
        'goal': 'maximize'   
        },
    'parameters' : {
        'optimizer': {
            'values': ['sgd']
            },
        'dropout': {
            'values': [0.1,0.2,0.3,0.4]
            },
        'learning_rate': {
            'distribution': 'uniform',
            'min': 3e-8,
            'max': 1e-7
            },
        'epochs': {
            'values': [4,5,6,7,8]
            },
        'max_len' : {
            'values' : [60,65,70,75,80]
            },
        }
    }

---
# 학습용 데이터셋 전처리
---

In [12]:
train_data = pd.read_csv('/content/drive/MyDrive/Kobert/data/Final_Train_dataset_0915.csv')
del train_data['Unnamed: 0']
train_data

,review,taste,quantity,delivery
0,양도 좋고 맛도 좋습니다 배달도 빨라요,1,1,1
1,양이많고 배달도 빠르네요 전 국물떡볶이가 맛있었답니다,1,1,1
2,너무 맛있어용 서비스로 주신 분모자도 감사합니다 아래 도어 번호를 안 써서 못들어 ...,1,1,1
3,사진을 드럽게 못 찍긴 했지만 배달 양 맛 싹 다 넘모 죠습니다,1,1,1
4,배달 완전 정확하게 오고 맛도 양도 넘 좋아요 맛있게 잘 먹었습니다,1,1,1
...,...,...,...,...
41682,하루두끼먹는데 그중 한끼,0,0,0
41683,원래 옆동네플랫폼에만 잇던거같은데 여기도 올라와서 좋네요 계속 자주먹을게요 ㅎㅎ,0,0,0
41684,다른 집 들렸다가 온 거 같네요 배달이 너무 늦어요,0,0,1
41685,익스프레스인데 바로 안옴,0,0,1


In [13]:
test_data = pd.read_csv('/content/drive/MyDrive/Kobert/data/Fianl_Test_dataset_0915.csv')
test_data

,review,taste,quantity,delivery
0,처음시켜봤는데 맛있고 빠른배달에 서비스 스파게티도 잘억었습니다,1,1,1
1,맛 좋고 빠르고 양많고,1,1,1
2,맛있고 배달 빠르고 가성비 좋아요 서비스도 굳,1,1,1
3,아주 맛있게 잘 먹었습니다 양도 많고 따뜻하게 배달되어서 좋았습니다,1,1,1
4,맛 훌륭함 양 적당함 배달 매우 빠름 p s 매번 리뷰 이벤트를 까먹는 것 같은 기...,1,1,1
...,...,...,...,...
1094,언제 먹어도 후회가 없어요 이벤트도 감사합니다,0,0,0
1095,순삭이에요 애들점심으로 줬더니 내꺼로 남을게 없네요ㅠ 다음에 또 주문할께요,0,0,0
1096,피자를 오랜만에 먹었네요 다른 곳보다 저렴하게 판매하셔서 감사합니다,0,0,0
1097,불편함없이 잘먹었습니다,0,0,0


In [14]:
import numpy as np 

total_data_text = list(train_data['review'])
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

문장길이 평균 :  29.730755391369012
문장길이 최대 :  263
문장길이 표준편차 :  20.815776882919657
pad_sequences maxlen :  71
전체 문장의 0.9373425768225107%가 maxlen 설정값 이내에 포함됩니다. 


---
# 모델
---

In [15]:
class BERTDataset(Dataset): 
  def __init__(self, dataset, bert_tokenizer, max_len):
    
    self.sentences = [bert_tokenizer(i, truncation=True, return_token_type_ids=True, padding="max_length",
                                     max_length = max_len, return_tensors = 'pt') for i in dataset['review']]
    self.max_len = max_len
    self.taste = [np.int64(i) for i in dataset['taste']]
    self.quantity = [np.int64(i) for i in dataset['quantity']]
    self.delivery = [np.int64(i) for i in dataset['delivery']]
  
  def __getitem__(self,i):
    self.input_ids = self.sentences[i]['input_ids']
    self.attention_mask_token = self.sentences[i]['attention_mask']
    self.token_type_ids = self.sentences[i]['token_type_ids']

    return self.input_ids, self.attention_mask_token, self.token_type_ids, self.taste[i], self.quantity[i], self.delivery[i]

  def __len__(self):
    return (len(self.taste))

In [16]:
# 하이퍼파라미터 세팅
max_len = 71
batch_size = 32
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate =  2e-6

In [17]:
data_train = BERTDataset(train_data, tokenizer, max_len)
data_test = BERTDataset(test_data, tokenizer, max_len)

In [18]:
train_dataloader = torch.utils.data.DataLoader(data_train, shuffle=True, batch_size=batch_size, num_workers=3, drop_last=True)
test_dataloader = torch.utils.data.DataLoader(data_test, shuffle=False, batch_size=batch_size, num_workers=3, drop_last=True)

In [19]:
# wandb.init(project = 'klue_bert_TAPT_before',
#                  entity="glad415",
#                  config = {
#                      "hidden_size" : 768,
#                      "max_len" : max_len,
#                      "batch_size" : batch_size,
#                      "learning_rate" : learning_rate,
#                      "epochs" : num_epochs,
#                      "optimizer" : 'adamW',
#                      "loss" : "BCEloss",
#                  })

In [20]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 ################# 클래스 수 조정 ##################
                 num_classes=1,   
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.taste = nn.Linear(hidden_size , num_classes)
        self.quantity = nn.Linear(hidden_size , num_classes)
        self.delivery = nn.Linear(hidden_size , num_classes)

        self.dropout = nn.Dropout(p=dr_rate)

        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask, token_type_ids):

        out = self.bert(input_ids = input_ids.long(), token_type_ids = token_type_ids.long(), attention_mask = attention_mask.to(device))
        out = self.dropout(out.pooler_output)

        taste_out = self.taste(out)
        quantity_out = self.quantity(out)
        delivery_out = self.delivery(out)   

        return self.sigmoid(taste_out), self.sigmoid(quantity_out), self.sigmoid(delivery_out)
    
    ############################# Frezee #############################
    # def freeze_bert_encoder(self):
    #     for param in self.bert.parameters():
    #         param.requires_grad = False
    
    # def unfreeze_bert_encoder(self):
    #     for param in self.bert.parameters():
    #         param.requires_grad = True
    ##################################################################

In [21]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.2).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.8)
loss_fn = nn.BCELoss() 

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

print(warmup_step)
print(t_total)

260
2604


In [22]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.8)
    
    elif optimizer == "adamW":
        optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
        
    return optimizer

In [23]:
# def custom_micro_f1(pred, label):
#   label_list = ['맛','양','배달']
#   label_indices = list(range(len(label_list)))
#   return f1_score(pred, label, labels= label_indices, average = 'micro')

In [24]:
def train(model, train_dataloader, loss_fn, optimizer, device, config, wandb):
  
  model.train()
  wandb.watch(model, criterion = loss_fn, log = 'all', log_freq=100)
  for e in range(config.epochs):

      train_acc = 0.0
      total_f1 = 0.0
      for batch_id, (input_ids, attention_mask_token, token_type_ids, taste_label, quantity_label, delivery_label) in enumerate(tqdm_notebook(train_dataloader)):
          
          optimizer.zero_grad()
          
          input_ids = input_ids.long().to(device)
          input_ids = input_ids.squeeze(1)

          token_type_ids = token_type_ids.long().to(device)
          token_type_ids = token_type_ids.squeeze(1)
          
          attention_mask_token= attention_mask_token.squeeze(1)

          taste_label = taste_label.float().to(device)
          taste_label = taste_label.unsqueeze(1)

          quantity_label = quantity_label.float().to(device)
          quantity_label = quantity_label.unsqueeze(1)

          delivery_label = delivery_label.float().to(device)
          delivery_label = delivery_label.unsqueeze(1)

          taste_out, quantity_out, delivery_out = model(input_ids, attention_mask_token, token_type_ids)

          # Loss
          ################################################################################################
          taste_loss = loss_fn(taste_out, taste_label)
          quantity_loss = loss_fn(quantity_out, quantity_label)
          delivery_loss = loss_fn(delivery_out, delivery_label)

          total_loss = (taste_loss + quantity_loss + delivery_loss) / 3
          total_loss.backward() 
          ################################################################################################

          torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

          optimizer.step()
          scheduler.step()  # Update learning rate schedule

          # Accuracy
          ################################################################################################
          taste_preds = (taste_out >= 0.5).float()
          quantity_preds = (quantity_out >= 0.5).float()
          delivery_preds = (delivery_out >= 0.5).float()

          taste_accuracy = (taste_preds == taste_label).cpu().numpy().mean()
          quantity_accuracy = (quantity_preds == quantity_label).cpu().numpy().mean()
          delivery_accuracy = (delivery_preds == delivery_label).cpu().numpy().mean()

          train_acc += (taste_accuracy + quantity_accuracy + delivery_accuracy) / 3
          ################################################################################################

          # F1_Score
          ################################################################################################
          taste_f1 = f1_score(taste_preds.cpu().data, taste_label.cpu(), average = 'micro')
          quantity_f1 = f1_score(quantity_preds.cpu().data, quantity_label.cpu(), average = 'micro')
          delivery_f1 = f1_score(delivery_preds.cpu().data, delivery_label.cpu(), average = 'micro')
          
          total_f1 +=  (taste_f1 + quantity_f1 + delivery_f1) / 3
          ################################################################################################

          if batch_id % log_interval == 0:

              # print(f"epoch {e+1} batch id {batch_id+1} loss {total_loss.data.cpu().numpy():.3f} train acc {train_acc / (batch_id+1):.3f}")
              # print(f'taste_accuracy {taste_accuracy:.3f}  | quantity_accuracy {quantity_accuracy:.3f}  | delivery_accuracy {delivery_accuracy:.3f}')
              # print(f'f1_score:{total_f1 / (batch_id+1):.3f}')
              # print('#'*80)

              # WandB
              #############################################################################################################################
              wandb.log({'interval_Loss': total_loss.data.cpu().numpy(),
                          'interval_taste_Accuracy' :taste_accuracy, 'interval_quantity_Accuracy' : quantity_accuracy, 'interval_delivery_Accuracy' : delivery_accuracy,
                          'interval_taste_F1' : taste_f1, 'interval_quantity_F1' : quantity_f1, 'interval_delivery_F1' : delivery_f1})
              #############################################################################################################################

      print(f"epoch {e+1} epoch_Train_Accuracy {train_acc / (batch_id+1):.3f}, epoch_F1_score {total_f1 / (batch_id+1):.3f}")
      
      # WandB
      #############################################################################################################################
      wandb.log({'epoch_Train_Accuracy' : train_acc / (batch_id+1), 'epoch_F1_score' : total_f1 / (batch_id+1)})
      #############################################################################################################################

def valid(model, test_dataloader, device):
  model.eval()
  test_f1 = 0.0
  test_acc = 0.0
  for batch_id, (input_ids, attention_mask_token, token_type_ids, taste_label, quantity_label, delivery_label) in enumerate(tqdm_notebook(test_dataloader)):
      
      input_ids = input_ids.long().to(device)
      input_ids = input_ids.squeeze(1)

      token_type_ids = token_type_ids.long().to(device)
      token_type_ids = token_type_ids.squeeze(1)
      
      attention_mask_token= attention_mask_token.squeeze(1)

      taste_label = taste_label.float().to(device)
      taste_label = taste_label.unsqueeze(1)

      quantity_label = quantity_label.float().to(device)
      quantity_label = quantity_label.unsqueeze(1)

      delivery_label = delivery_label.float().to(device)
      delivery_label = delivery_label.unsqueeze(1)

      taste_out, quantity_out, delivery_out = model(input_ids, attention_mask_token, token_type_ids)

      # Test_Accuracy
      ################################################################################################    
      taste_preds = (taste_out >= 0.5).float()
      quantity_preds = (quantity_out >= 0.5).float()
      delivery_preds = (delivery_out >= 0.5).float()

      taste_accuracy = (taste_preds == taste_label).cpu().numpy().mean()
      quantity_accuracy = (quantity_preds == quantity_label).cpu().numpy().mean()
      delivery_accuracy = (delivery_preds == delivery_label).cpu().numpy().mean()

      test_acc += (taste_accuracy + quantity_accuracy + delivery_accuracy) / 3
      ################################################################################################ 

      # Test_F1_Score
      ################################################################################################
      taste_f1 = f1_score(taste_preds.cpu().data, taste_label.cpu(), average = 'micro')
      quantity_f1 = f1_score(quantity_preds.cpu().data, quantity_label.cpu(), average = 'micro')
      delivery_f1 = f1_score(delivery_preds.cpu().data, delivery_label.cpu(), average = 'micro')
      
      test_f1 +=  (taste_f1 + quantity_f1 + delivery_f1) / 3
      ################################################################################################

  print('-'*40,'Test_Value','-'*40)
  print(f"test_Accuracy {test_acc / (batch_id+1):.3f}, test_F1_score {test_f1/ (batch_id+1):.3f}")
  print('-'*92)

  # WandB
  #############################################################################################################################
  wandb.log({'Test_acc' : test_acc / (batch_id+1), 'Test_F1_Score' : test_f1/ (batch_id+1)})
  #############################################################################################################################

In [25]:
def run_sweeep(config=None):
  
    wandb.init(config=config)
    
    w_config = wandb.config

    data_train = BERTDataset(train_data, tokenizer, w_config.max_len)
    data_test = BERTDataset(test_data, tokenizer, w_config.max_len)

    train_dataloader = torch.utils.data.DataLoader(data_train, shuffle=True, batch_size=batch_size, num_workers=0, drop_last=True)
    test_dataloader = torch.utils.data.DataLoader(data_test, shuffle=False, batch_size=batch_size, num_workers=0, drop_last=True)

    model = BERTClassifier(bertmodel, dr_rate = w_config.dropout).to(device)
    optimizer = build_optimizer(model, w_config.optimizer, w_config.learning_rate)
    
    train(model, train_dataloader, loss_fn, optimizer, device, w_config, wandb)
    valid(model, test_dataloader, device)

In [26]:
sweep_id = wandb.sweep(sweep_config, project="klue_sweep", entity='glad415')
wandb.agent(sweep_id, run_sweeep, count=6)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: x6pwm4bl
Sweep URL: https://wandb.ai/glad415/klue_sweep/sweeps/x6pwm4bl


wandb: Agent Starting Run: 9n82v1qf with config:
wandb: 	dropout: 0.4
wandb: 	epochs: 5
wandb: 	learning_rate: 4.6609864559442804e-08
wandb: 	max_len: 65
wandb: 	optimizer: sgd


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/1302 [00:00<?, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/1302 [00:00<?, ?it/s]

wandb: Ctrl + C detected. Stopping sweep.


Error in callback <function _WandbInit._pause_backend at 0x7f48388a4710> (for post_run_cell):


BrokenPipeError: ignored

Error in callback <function _WandbInit._pause_backend at 0x7f48388a4710> (for post_run_cell):


BrokenPipeError: ignored